In [24]:
%pylab qt

import os
import itertools
import cv2
import numpy as np
from glob import glob
from scipy.interpolate import splprep, splev

import helpers.features as fh
import helpers.display as dh
import helpers.geometry as gh
reload(fh)
reload(dh)
reload(gh)

BASE_PATH = os.getcwd()
print("Current base path: {0}".format(BASE_PATH))
DATA_PATH = BASE_PATH + '/Daten/2D/Talus_dorsal_mesh/'
TO_PATH = BASE_PATH + '/Daten/2D/Talus_dorsal_registered_outline/'

NUM_SPLINE_POINTS = 250

def extract_spline(points):
    y = points[:,0].flatten()
    x = points[:,1].flatten()
    
    tck, u = splprep([y, x], s=0)
    coords = splev(np.linspace(0, 1, NUM_SPLINE_POINTS), tck)
    
    spline_points = np.zeros((NUM_SPLINE_POINTS, 2))
    spline_points[:, 0] = coords[0]
    spline_points[:, 1] = coords[1]
    
    return spline_points

def register_outline_with_icp(reference_points, points, edges, no_iterations = 1):
    ref_spline = extract_spline(reference_points)
    
    for i in range(no_iterations):
        #points, edges = gh.normalize_outline(points, edges)
        spline_points = extract_spline(points)
        
        R, t = gh.estimate_rigid_transform(spline_points, ref_spline)
        R = np.array(R)
        t = np.array(t)
        #Transform the previous source and update the
        #current source cloudpoint
        points = (np.dot(R, points.transpose())).transpose() + np.tile(t, (points.shape[0], 1))
    
    return points

def do_registration(files):
    loaded = []
    transposed_and_scaled = []
    registered = []
    
    for f in files:
        tri = np.load(f)
        points = tri['points']
        triangles = tri['triangles']
        
        label = os.path.basename(f)
        label = ''.join([i if ord(i) < 128 else ' ' for i in label])
        
        outline_points, outline_edges = gh.extract_outline(points, tri['triangles'])
        
        loaded.append({
            'label': label,
            'filename': f,
            'points': outline_points,
            'edges': outline_edges
        })
           
    for outline in loaded:
        points = outline['points']
        
        centroid = mean(points, axis=0)
        points = points - np.tile(centroid, (len(points), 1))
        
        scale = sqrt(np.sum(np.power(points, 2)) / len(points))
        points = np.divide(points, scale)
        points[: ,0] = -points[:, 0]
        
        edges = outline['edges']
        points, edges = gh.normalize_outline(points, outline['edges'])
        
        transposed_and_scaled.append({
            'label': outline['label'],
            'filename': outline['filename'],
            'points': points,
            'edges': edges
        })
    
    temp = []
    for outline in transposed_and_scaled:
        points = extract_spline(outline['points'])
        edges = np.zeros((points.shape[0], 2), dtype=np.int)
        edges[:, 0] = range(0, points.shape[0])
        edges[:, 1] = range(1, points.shape[0]+1)
        edges[-1, 1] = 0
        #print(points)
        temp.append({
            'label': outline['label'],
            'filename': outline['filename'],
            'points': points,
            'edges': edges
        })
        
    reference = max(transposed_and_scaled, key=lambda o: o['points'].shape[0])
    reference_points = reference['points']
    reference_edges = reference['edges']
    for outline in transposed_and_scaled:
        points = outline['points']
        edges = outline['edges']
        
        if not np.array_equal(points, reference):
            points = register_outline_with_icp(reference_points, points, edges)
            points, edges = gh.normalize_outline(points, edges)
        registered.append({
            'label': outline['label'],
            'filename': outline['filename'],
            'points': points,
            'edges': edges
        })
    
    return registered

Populating the interactive namespace from numpy and matplotlib
Current base path: /home/stefan/Dropbox/Masterarbeit


In [15]:
registered = do_registration(glob(DATA_PATH + '*.npz'))

In [25]:
dh.outlines(plt, registered)

(<matplotlib.figure.Figure at 0x7f2b1ead4990>,
 <matplotlib.axes._subplots.AxesSubplot at 0x7f2b1ead4f90>)

In [7]:
temp = []
for outline in registered:
    points = extract_spline(outline['points'])
    edges = np.zeros((points.shape[0], 2), dtype=np.int)
    edges[:, 0] = range(0, points.shape[0])
    edges[:, 1] = range(1, points.shape[0]+1)
    edges[-1, 1] = 0
    #print(points)
    temp.append({
        'label': outline['label'],
        'filename': outline['filename'],
        'points': points,
        'edges': edges
    })
    temp.append({
        'label': outline['label'] + '_org',
        'filename': outline['filename'],
        'points': outline['points'],
        'edges': outline['edges']
    })
dh.outlines(plt, temp, show_direction=True)

(<matplotlib.figure.Figure at 0x30e9f978>,
 <matplotlib.axes._subplots.AxesSubplot at 0x3259a5f8>)

In [19]:
for outline in registered:
    basename = os.path.basename(outline['filename'])
    destination = os.path.join(TO_PATH, basename)
    
    np.savez(destination, points=outline['points'], edges=outline['edges'])